<a href="https://colab.research.google.com/github/AkiraNom/data-analysis-notebook/blob/main/Stock_analysis_Streamlit_Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
! pip install streamlit

In [32]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.674s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [33]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip('\n'))

Password/Enpoint IP for localtunnel is: 34.125.24.211


In [34]:
%%writefile utils.py

import pandas as pd
import yfinance as yf

# get stock price through yfinance
def get_stock_price(ticker_info):
# obtain last 2 years stock price for plot
# other valid period option: “1d”, “5d”, “1mo”,
#“3mo”, “6mo”, “1y”, “2y”, “5y”, “10y”, “ytd”, “max”
      return ticker_info.history(period='max')

# get cash flow data through yfinance
def get_cash_flow_data(ticker_info):

    cash_flow_yr = ticker_info.cash_flow
    cash_flow_yr.columns = cash_flow_yr.columns.year

    cash_flow_q = ticker_info.quarterly_cashflow
    col_names = [f'Q{q} {year}' for q, year in zip(cash_flow_q.columns.quarter, cash_flow_q.columns.year)]
    cash_flow_q.columns = col_names

    return cash_flow_yr, cash_flow_q

# get financial data through yfinance
def get_financial_data(ticker_info):

    financial_yr = ticker_info.financials
    financial_yr.columns = financial_yr.columns.year

    financial_q = ticker_info.quarterly_financials
    col_names = [f'Q{q} {year}' for q, year in zip(financial_q.columns.quarter, financial_q.columns.year)]
    financial_q.columns = col_names

    return financial_yr, financial_q

Overwriting utils.py


In [35]:
%%writefile plot_func.py

import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go

def plot_candle_chart(df):

  # roughly 250 trading days in one year
  yr = 250

  fig = go.Figure()

  # Add Traces
  fig.add_trace(
      go.Candlestick(x=df.index[-yr:],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    name='Candle_1y',
                    visible = True
                    ))

  fig.add_trace(
      go.Candlestick(x=df.index[-yr*2:],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    name='Candle_2y',
                    visible = False
                    ))

  fig.add_trace(
      go.Candlestick(x=df.index[-yr*5:],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    name='Candle_5y',
                    visible = False
                    ))

  fig.add_trace(
      go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    name='Candle_max',
                    visible = False
                    ))



  fig.update_layout(
      updatemenus=[
          dict(
              type="buttons",
              direction="right",
              active=0,
              x=0.57,
              y=1.2,
              buttons=[
                  dict(label="1yr",
                      method="update",
                      args=[{"visible": [True,False,False,False]}],
                      ),
                  dict(label="2yr",
                      method="update",
                      args=[{"visible": [False,True,False,False]}],
                      ),
                  dict(label="5yr",
                      method="update",
                      args=[{"visible": [False,False,True,False]}],
                      ),
                  dict(label="Max",
                      method="update",
                      args=[{"visible": [False,False,False,True]}]
                      ),
                  ]
              )
          ]
      )

  return fig

def plot_bar_chart(df_yr, df_q, vars_cat, document='Cash Flow'):

  fig_yr = go.Figure()
  fig_q = go.Figure()

  for var_cat in vars_cat:
    if 'Cash Flow' in var_cat:
      name = var_cat.replace('Cash Flow From Continuing ' ,'')
      name = name.replace('Activities','')
    elif 'Total' in var_cat:
      name = var_cat.replace('Total ', '')
    else:
      name = var_cat.replace('Net Income Continuous Operations','Earnings')


    fig_yr.add_trace(go.Bar(name = name,
                            x=df_yr.columns,
                            y=df_yr.loc[var_cat,:]))

    fig_q.add_trace(go.Bar(name = name,
                            x=df_q.columns,
                            y=df_q.loc[var_cat,:]))

  fig_yr.update_layout(xaxis={'categoryorder':'total descending'})
  fig_q.update_layout(xaxis={'categoryorder':'total ascending'})

  # place the legend horizontally
  fig_yr.update_layout(legend=dict(
                                orientation="h",
                                entrywidth=100,
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                        margin=dict(
                                l=50,
                                r=50,
                                b=30,
                                t=30,
                                pad=4
    ),)

  fig_q.update_layout(legend=dict(
                              orientation="h",
                              entrywidth=100,
                              yanchor="bottom",
                              y=1.02,
                              xanchor="right",
                              x=1),
                        margin=dict(
                              l=50,
                              r=50,
                              b=30,
                              t=30,
                              pad=4
    ))


  return fig_yr, fig_q

def plot_scatter(df_yr, df_q, vars_cat):

  fig_yr = go.Figure()
  fig_q = go.Figure()

  colors = ['rgba(204, 204, 204, 0.95)', 'rgba(98,98,226,0.5)']


  for var_cat, color in zip(vars_cat, colors):
    if 'Total' in var_cat:
      name = var_cat.replace('Total ', '')
    else:
      name = var_cat.replace('Net Income Continuous Operations','Earnings')

    fig_yr.add_trace(go.Scatter(name = name,
                            x=df_yr.columns,
                            y=df_yr.loc[var_cat,:],
                            marker=dict(color=color, line_color=color),
                            mode='markers'))

    fig_q.add_trace(go.Scatter(name = name,
                            x=df_q.columns,
                            y=df_q.loc[var_cat,:],
                            marker=dict(color=color, line_color=color),
                            mode='markers'))

    fig_yr.update_traces(mode='markers', marker=dict(line_width=1, symbol='circle', size=16))

    fig_q.update_traces(mode='markers', marker=dict(line_width=1, symbol='circle', size=16))

  return fig_yr, fig_q

def plot_sankey_diagram(df, link, report_name):

  link = dict(source = df['source'],
              target = df['target'],
              value = df['value'],
              color=df['link_colors'])

  node = dict(label = df['label'],
              x = df['x'],
              y =df['y'] ,
              color=df['colors'],
              pad=20,
              thickness=10)

  data = go.Sankey(link = link, node=node)

  # plot
  fig = go.Figure(data)
  fig.update_layout(
  title_text=report_name,
  font_family="Arial Black",
  font_color="Black",
  font_size=11,)

  note = f'Source:<a href="https://www.sec.gov/"">The SEC</a> Data: <a href=link>{report_name}</a>'
  fig.add_annotation(
      showarrow=False,
      text=note,
      font=dict(size=10),
      xref='x domain',
      x=0.0,
      yref='y domain',
      y=-0.2
      )

  return fig

Overwriting plot_func.py


In [36]:
%%writefile sankey_data.py

import pandas as pd

def obtain_tesla_data():
  # Data from TESLA Q3 FY23 report
  data = {
  'labels':['Automotive sales', 'Automotive regulatory credits', 'Automotive leasing','Automotive revenue',
            'Energy generation & storage','Services','Revenue','Gross profit',
            'Cost of revenue','Operating profit', 'Operating expense', 'Net profit',
            'Tax','Other','Interest','Automotive sales',
            'Automotive leasing', 'Energy generation & storage', 'Services', 'Research & development',
            'Selling, general & admit'],

  # where link starts
  'source':[0,1,2,3,
            4,5,3,6,
            6,7,7,9,
            9,9,14,8,
            8,8,8,10,10],

  # where link ends
  'target':[3,3,3,6,
            6,6,6,7,
            8,9,10,11,
            12,13,11,15,
            16,17,18,19,20],

  # item values
  'value':[18582,554,489,19625,
           1559,2166,0,4178,
           19172,1764,2414,1878,
           37,167,282,15656,
           301,1178,2037,1161,1253],

  # define the link color
  'link_colors':['rgb(140,140,140)','rgb(140,140,140)','rgb(140,140,140)','rgb(140,140,140)',
                'rgb(140,140,140)','rgb(140,140,140)','rgb(140,140,140)','rgb(153,162,231)',
                'rgb(240,128,128)','rgb(153,162,231)','rgb(240,128,128)','rgb(153,162,231)',
                'rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)',
                'rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)'],

  # separate the item color
  'colors': ['rgb(0,0,0)','rgb(0,0,0)','rgb(0,0,0)','rgb(0,0,0)',
             'rgb(0,0,0)','rgb(0,0,0)','rgb(0,0,0)','rgb(0,31,231)',
             'rgb(231,41,0)','rgb(0,31,231)','rgb(231,41,0)','rgb(0,31,231)',
             'rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)',
             'rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)'],

  # define node position (x,y)
  'x':[0,0.05,0.05,0.1,
       0.25,0.25,0.2,0.5,
       0.5,0.6,0.6,0.8,
       0.65,0.65,0.75,0.6,
       0.6,0.6,0.6,0.75,0.75],

  'y':[0,0.8,0.9,0,
       0.9,1,0,0.3,
       0.7,0.1,0.4,0.1,
       0.2,0.28,0.2,0.7,
       0.9,1,1.1,0.4,0.5]
  }

  link = "https://www.sec.gov/edgar/browse/?CIK=1318605&owner=exclude/"
  report_name = 'TESLA Q3 FY23'

  return data, link, report_name

def tesla_sankey_data():

  tesla_data,link,report_name = obtain_tesla_data()
  df = pd.DataFrame(tesla_data)

  # sum up revenue
  val = (df[df['labels']=='Automotive revenue']['value'].values[0]+
        df[df['labels']=='Energy generation & storage']['value'].values[0]+
        df[df['labels']=='Services']['value'].values[0])/1000

  df.loc[:,'label'] = [f'{label} <br>{v/1000}B' if v != 0 else f'{label}<br>{val}B' for label, v in zip(df['labels'], df['value'])]

  return df, link, report_name

def obtain_apple_data():
  # Data from AAPL Q4 FY23 report
  data = {
  'labels':['Products','Services','Net revenue',
            'Cost of sales','Products','Services',
            'Gross profit','Operating profit','Operating expenses',
            'Research and Development','Selling, general and administrative','Tax','Other',
            'Net income'],

  # where link starts
  'source':[0,1,2,
            2,3,3,
            2,6,6,
            8,8,7,
            7,7,],

  # where link ends
  'target':[2,2,2,
            3,4,5,
            6,7,8,
            9,10,11,
            12,13,],

  # item values
  'value':[96458, 23117, 0,
           64720, 58440, 6280,
           54855, 40373, 14482,
           7696, 6786, 6407,
           50, 33916],

  # define the link color
  'link_colors':['rgb(140,140,140)','rgb(140,140,140)','rgb(140,140,140)',
                'rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)',
                'rgb(153,162,231)','rgb(153,162,231)','rgb(240,128,128)',
                'rgb(240,128,128)','rgb(240,128,128)','rgb(240,128,128)',
                 'rgb(240,128,128)','rgb(153,162,231)'],

  # separate the item color
  'colors': ['rgb(0,0,0)','rgb(0,0,0)','rgb(0,0,0)',
             'rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)',
             'rgb(0,31,231)','rgb(0,31,231)','rgb(231,41,0)',
             'rgb(231,41,0)','rgb(231,41,0)','rgb(231,41,0)',
             'rgb(231,41,0)','rgb(0,31,231)'],

  # define node position (x,y)
  'x':[0,0,0,
       0.4,0.5,0.5,
       0.4,0.5,0.5,
       0.65, 0.65,0.65,
       0.65,0.7],

  'y':[0,0.2,0,
       0.8,0.8,1,
       0.3,0.2,0.5,
       0.5,0.6,-0.05,
       0.05,0.2]
  }

  link = "https://www.sec.gov/edgar/browse/?CIK=320193&owner=exclude"
  report_name = 'APPLE Q4 FY23'

  return data, link, report_name


def apple_sankey_data():

  apple_data,link,report_name = obtain_apple_data()
  df = pd.DataFrame(apple_data)

  # sum up revenue
  val = (df[df['labels']=='Products']['value'].values[0]+
        df[df['labels']=='Services']['value'].values[0])/1000

  df.loc[:,'label'] = [f'{label} <br>{v/1000}B' if v != 0 else f'{label}<br>{val}B' for label, v in zip(df['labels'], df['value'])]

  return df, link, report_name


Overwriting sankey_data.py


In [37]:
%%writefile app.py

import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import streamlit as st
import yfinance as yf

import sankey_data
import plot_func
import utils

st.set_page_config(layout="wide")

# Layout (Sidebar)

st.sidebar.markdown("## Data Load")
# select the company

default_value = 'TSLA'

TICKER = st.sidebar.text_input(
    label = 'Type a Ticker Symbol',
    value = default_value
)

# select form Yearly or Quarterly
#PERIOD  = st.sidebar.selectbox(
#    label = 'Select Period of Financial Data',
#    options = ('Yearly','Quarterly'),
#    placeholder = 'Yearly'
#)

action_button = st.sidebar.button(
    label = 'Run'
)

st.sidebar.write('Sankey diagrams are available for Tesla or Apple')

if action_button == True:

    main_title = f'{TICKER} Stock Data Analysis'
    st.title(main_title)

    TICKER_INFO = yf.Ticker(TICKER)

 # add error handling

    stock_price = utils.get_stock_price(TICKER_INFO)
    cash_flow_yr, cash_flow_q = utils.get_cash_flow_data(TICKER_INFO)
    financial_yr, financial_q = utils.get_financial_data(TICKER_INFO)

    #Extract variable for cash flow bar plot
    prefixes = ['Cash Flow','Free Cash']
    vars_cat = [var for var in cash_flow_yr.index if var.startswith(tuple(prefixes))]

    # variables for earnigns and Revenue
    vars_cat_earning_revenue = ['Net Income Continuous Operations', 'Total Revenue']

    # Layout (Content)
    # stock price (top row)
    top_row_subheader = 'Stock Price'
    st.subheader(top_row_subheader)
    st.plotly_chart(plot_func.plot_candle_chart(stock_price), use_container_width=True)

    left_column, right_column = st.columns(2)

    # bar plot (left column)
    left_column_subheader = 'Cash Flow'
    left_column.subheader(left_column_subheader)

    fig_yr, fig_q = plot_func.plot_bar_chart(cash_flow_yr, cash_flow_q, vars_cat)
    tab1, tab2 = left_column.tabs(["Yearly", "Quarterly"])
    with tab1:
        st.plotly_chart(fig_yr, theme=None, use_container_width=True)
    with tab2:
        st.plotly_chart(fig_q, theme=None, use_container_width=True)

    # bar plot (right column)
    right_column_subheader = 'Revenue & Earnings'
    right_column.subheader(right_column_subheader)

    vars_cat = ['Net Income Continuous Operations', 'Total Revenue']

    fig_yr, fig_q = plot_func.plot_scatter(financial_yr, financial_q, vars_cat)
    tab1, tab2 = right_column.tabs(["Yearly", "Quarterly"])
    with tab1:
        st.plotly_chart(fig_yr, theme=None, use_container_width=True)
    with tab2:
        st.plotly_chart(fig_q, theme=None, use_container_width=True)


    # generate sankey diagram
    if TICKER_INFO.ticker == 'TSLA':
      df, link, report_name = sankey_data.tesla_sankey_data()
      st.plotly_chart(plot_func.plot_sankey_diagram(df,link, report_name),use_container_width=True)

    if TICKER_INFO.ticker == 'AAPL':
      df, link, report_name = sankey_data.apple_sankey_data()
      st.plotly_chart(plot_func.plot_sankey_diagram(df,link, report_name),use_container_width=True)


Overwriting app.py


In [38]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501

npx: installed 22 in 2.55s
your url is: https://rotten-tigers-cough.loca.lt
^C
